In [356]:
import numpy as np
import os
import cv2
import matplotlib
import matplotlib.pyplot as plt
import echonet

# matplotlib.use('agg')

In [357]:
def loadvideo(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError()
    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # empty numpy array of appropriate length, fill in when possible from front
    v = np.zeros((frame_count, frame_width, frame_height, 3), np.float32)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        v[count] = frame

    v = v.transpose((3, 0, 1, 2))

    return v

In [358]:
videoid = "0X22E82C3D081C819C"
path_base = '/home/jovyan/code/EchoNetDynamic/output/segmentation/deeplabv3_resnet50_random/labels'
seg = np.load(os.path.join(path_base, videoid + ".npy"))

In [359]:
seg.shape

(172, 112, 112)

In [360]:
video_path = '/home/jovyan/data/EchoNet-Dynamic/Videos/' + videoid + '.avi'
video = loadvideo(video_path)

In [361]:
video.shape

(3, 172, 112, 112)

In [362]:
# plt.imshow(video[:,0,...].transpose(1,2,0).astype(np.uint8))
# plt.imshow(video)
# plt.savefig('test.png')

In [363]:
echonet.config.DATA_DIR = '../../data/EchoNet-Dynamic'
mean, std = echonet.utils.get_mean_and_std(echonet.datasets.Echo(split="train"))

100%|██████████| 10/10 [00:00<00:00, 30.73it/s]


In [364]:
if isinstance(mean, int) or isinstance(mean, float):
    video = (video - mean) / std
else:
    video = (video - mean.reshape(3, 1, 1, 1)) / std.reshape(3, 1, 1, 1)


In [365]:
tmp = video[:,0,...].transpose(1,2,0).copy()
tmp = (tmp - tmp.min())/(tmp.max()-tmp.min())
cv2.imwrite('test_normalized.png', np.uint8(tmp*255)) 
video = video- video.mean()
tmp = video
video[tmp!=0] = -video[tmp!=0]
video = (video - video.min())/(video.max()-video.min())
# # plt.imshow(np.uint8(video[:,0,...].transpose(1,2,0)*255))
# # plt.savefig('test_normalized.png')
cv2.imwrite('test_normalized_inverse.png', np.uint8(video[:,0,...].transpose(1,2,0)*255)) 

True

In [353]:
tmp.shape

(3, 172, 112, 112)

In [355]:
alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)
tmp = video[:,0,...].transpose(1,2,0).copy()
adjusted = cv2.convertScaleAbs(tmp , alpha=alpha, beta=beta)
adjusted = (adjusted - adjusted.min())/(adjusted.max()-adjusted.min())
cv2.imwrite('adjusted_normalized.png', np.uint8(adjusted*255))

True

In [347]:
print(adjusted.min(),adjusted.max())

0 255


In [350]:
adjusted.shape

(3, 172, 112, 112)

In [322]:
print(seg.max(),seg.min())

1.0 0.0


In [323]:
seg[seg<0] = 0
seg = seg/seg.max()

In [324]:
# seg = seg[..., np.newaxis]
# plt.imshow(seg[0,...])
# plt.savefig('seg.png')
for i in range(10):
    cv2.imwrite('seg_{}.png'.format(i), np.uint8(seg[i,...]*255)) 

In [325]:
def seg_inverse(seg,video,name):
    seg_mask = np.zeros((112,112,3))
    video_out = np.zeros((112,112,3))
    video_out_blur = np.zeros((112,112,3))
    video_tmp = video.transpose(1,2,0)
    for i in range(3):
    #     seg_mask[...,i] = seg[0,...]
        seg_tmp = cv2.GaussianBlur(np.uint8(seg*255),(31,31),5)
        video_out[...,i] = seg*video_tmp[...,i]/seg.sum()
        video_out_blur[...,i] = seg_tmp*video_tmp[...,i]/seg_tmp.sum()
    video_out = (video_out - video_out.min())/ (video_out.max()-video_out.min())
    video_out_blur = (video_out_blur - video_out_blur.min())/ (video_out_blur.max()-video_out_blur.min())
    cv2.imwrite(name + '_test_normalized_inverse_seg.png', np.uint8(video_out*255)) 
    cv2.imwrite(name +'_test_normalized_inverse_seg_blur.png', np.uint8(video_out_blur*255)) 

seg_inverse(seg[0,...],video[:,0,...],'seg')

In [326]:
import collections
def _defaultdict_of_lists():
    return collections.defaultdict(list)
frames = collections.defaultdict(list)
trace = collections.defaultdict(_defaultdict_of_lists)
folder = '/home/jovyan/data/EchoNet-Dynamic/'
with open(folder + "VolumeTracings.csv") as f:
    header = f.readline().strip().split(",")
    for (i, line) in enumerate(f):
        filename, x1, y1, x2, y2, frame = line.strip().split(',')
        x1 = float(x1)
        y1 = float(y1)
        x2 = float(x2)
        y2 = float(y2)
        frame = int(frame)
        if frame not in trace[filename]:
            frames[filename].append(frame)
        trace[filename][frame].append((x1, y1, x2, y2))
for filename in frames:
    for frame in frames[filename]:
        trace[filename][frame] = np.array(trace[filename][frame])

In [327]:
large = trace[videoid][frames[videoid][-1]]
small = trace[videoid][frames[videoid][0]]

In [328]:
import skimage
def draw_mask(t):
    x1, y1, x2, y2 = t[:, 0], t[:, 1], t[:, 2], t[:, 3]
    x = np.concatenate((x1[1:], np.flip(x2[1:])))
    y = np.concatenate((y1[1:], np.flip(y2[1:])))
    r, c = skimage.draw.polygon(np.rint(y).astype(np.int), np.rint(x).astype(np.int), (video.shape[2], video.shape[3]))
    mask = np.zeros((video.shape[2], video.shape[3]), np.float32)
    mask[r, c] = 1
    return mask

In [329]:
large = draw_mask(large)
small = draw_mask(small)

In [330]:
seg_inverse(large,video[:,31,...],'large')
seg_inverse(small,video[:,2,...],'small')

In [331]:
# cv2.imwrite('large.png', np.uint8(large*255)) 
# cv2.imwrite('small.png', np.uint8(small*255)) 

In [332]:
logit = seg[:32,...]
size = (logit > 0).sum(2).sum(1)

In [333]:
size

array([1515, 1699, 1933, 1769, 1766, 1905, 1908, 1594, 1408, 1246, 1367,
       1173, 1076, 1175, 1211, 1187, 1262, 1337, 1432, 1433, 1347, 1315,
       1235, 1280, 1143, 1270, 1198, 1177, 1031, 1045, 1020,  994])

In [334]:
print(size.argmin(),size.argmax())

31 2


In [335]:
seg_inverse(logit[31,...],video[:,31,...],'large_seg')
seg_inverse(logit[2,...],video[:,2,...],'small_seg')